In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import lpips
import os
import imageio
import numpy as np

In [1]:
# 均方误差（Mean Square Error）类
class MSE(object):
    def __call__(self, pred, gt):
        return torch.mean((pred - gt) ** 2)

In [3]:
# 峰值信噪比（Peak Signal to Noise Ratio）类
class PSNR(object):
    def __call__(self, pred, gt):
        mse = torch.mean((pred - gt) ** 2)
        return 10 * torch.log10(1 / mse)

In [4]:
# 结构相似性指数（Structural Similarity Index，SSIM）类
class SSIM(object):
    '''
    borrowed from https://github.com/huster-wgm/Pytorch-metrics/blob/master/metrics.py
    '''
    def gaussian(self, w_size, sigma):
        # 创建高斯窗函数
        gauss = torch.Tensor([math.exp(-(x - w_size//2)**2/float(2*sigma**2)) for x in range(w_size)])
        return gauss/gauss.sum()

    def create_window(self, w_size, channel=1):
        # 创建二维高斯窗口
        _1D_window = self.gaussian(w_size, 1.5).unsqueeze(1)
        _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
        window = _2D_window.expand(channel, 1, w_size, w_size).contiguous()
        return window

    def __call__(self, y_pred, y_true, w_size=11, size_average=True, full=False):
        """
        args:
            y_true : 4-d ndarray in [batch_size, channels, img_rows, img_cols]
            y_pred : 4-d ndarray in [batch_size, channels, img_rows, img_cols]
            w_size : int, default 11
            size_average : boolean, default True
            full : boolean, default False
        return ssim, larger the better
        """
        # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
        # 根据图像的最大值来确定最大值范围
        if torch.max(y_pred) > 128:
            max_val = 255
        else:
            max_val = 1

        # 根据图像的最小值来确定最小值范围
        if torch.min(y_pred) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val

        padd = 0
        (_, channel, height, width) = y_pred.size()
        window = self.create_window(w_size, channel=channel).to(y_pred.device)

        # 计算预测图像和真实图像的局部均值
        mu1 = F.conv2d(y_pred, window, padding=padd, groups=channel)
        mu2 = F.conv2d(y_true, window, padding=padd, groups=channel)

        # 计算均值的平方和
        mu1_sq = mu1.pow(2)
        mu2_sq = mu2.pow(2)
        mu1_mu2 = mu1 * mu2

        # 计算方差
        sigma1_sq = F.conv2d(y_pred * y_pred, window, padding=padd, groups=channel) - mu1_sq
        sigma2_sq = F.conv2d(y_true * y_true, window, padding=padd, groups=channel) - mu2_sq
        sigma12 = F.conv2d(y_pred * y_true, window, padding=padd, groups=channel) - mu1_mu2

        C1 = (0.01 * L) ** 2 # 常数C1
        C2 = (0.03 * L) ** 2 # 常数C2

        # 计算对比度敏感度
        v1 = 2.0 * sigma12 + C2
        v2 = sigma1_sq + sigma2_sq + C2
        cs = torch.mean(v1 / v2)  # contrast sensitivity

        # 计算结构相似性指数
        ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

        if size_average:
            # 如果需要平均，返回平均的SSIM值
            ret = ssim_map.mean()
        else:
            # 否则返回每个通道的SSIM均值
            ret = ssim_map.mean(1).mean(1).mean(1)

        if full:
            return ret, cs # 返回完整的结构相似性指数和对比度敏感度
        return ret # 返回SSIM值


In [5]:
# 学习感知图像补丁相似性（LPIPS）类
class LPIPS(object):
    '''
    borrowed from https://github.com/huster-wgm/Pytorch-metrics/blob/master/metrics.py
    '''
    def __init__(self):
        self.model = lpips.LPIPS(net='vgg').cuda()

    def __call__(self, y_pred, y_true, normalized=True):
        """
        args:
            y_true : 4-d ndarray in [batch_size, channels, img_rows, img_cols]
            y_pred : 4-d ndarray in [batch_size, channels, img_rows, img_cols]
            normalized : change [0,1] => [-1,1] (default by LPIPS)
        return LPIPS, smaller the better
        """
        if normalized:
            # 如果需要归一化，将值从[0,1]转换为[-1,1]
            y_pred = y_pred * 2.0 - 1.0
            y_true = y_true * 2.0 - 1.0
        error =  self.model.forward(y_pred, y_true)
        return torch.mean(error)

In [6]:
# 从指定目录读取所有图像
def read_images_in_dir(imgs_dir):
    imgs = []
    fnames = os.listdir(imgs_dir)
    fnames.sort()
    for fname in fnames:
        if fname == "000.png" :  # ignore canonical space, only evalute real scene
            continue
            
        img_path = os.path.join(imgs_dir, fname)
        img = imageio.imread(img_path)
        img = (np.array(img) / 255.).astype(np.float32)
        img = np.transpose(img, (2, 0, 1))
        imgs.append(img)
    
    imgs = np.stack(imgs)       
    return imgs

# 计算预测图像与真实图像的多种误差
def estim_error(estim, gt):
    errors = dict()
    metric = MSE()
    errors["mse"] = metric(estim, gt).item()
    metric = PSNR()
    errors["psnr"] = metric(estim, gt).item()
    metric = SSIM()
    errors["ssim"] = metric(estim, gt).item()
    metric = LPIPS()
    errors["lpips"] = metric(estim, gt).item()
    return errors

# 将误差保存到指定目录
def save_error(errors, save_dir):
    save_path = os.path.join(save_dir, "metrics.txt")
    f = open(save_path,"w")
    f.write( str(errors) )
    f.close()

In [7]:
files_dir = "./logs/mutant/renderonly_test_799999/" # 定义图像文件的路径

estim_dir = os.path.join(files_dir, "estim") # 预测图像目录
gt_dir = os.path.join(files_dir, "gt") # 真实图像目录

# 读取预测图像和真实图像
estim = read_images_in_dir(estim_dir)
gt = read_images_in_dir(gt_dir)

# 将图像转换为Tensor并转到GPU
estim = torch.Tensor(estim).cuda()
gt = torch.Tensor(gt).cuda()

errors = estim_error(estim, gt)
save_error(errors, files_dir)
print(errors)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/apumarola/miniconda3/envs/nerf/lib/python3.6/site-packages/lpips/weights/v0.1/vgg.pth
{'mse': 0.0007424007053487003, 'psnr': 31.293617248535156, 'ssim': 0.9739180207252502, 'lpips': 0.026781609281897545}
